In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import gzip
import re
from collections import OrderedDict, Counter
import pickle
from tqdm import tqdm
import itertools
import glob

In [2]:
url = 'https://uk.wikipedia.org/w/index.php?title=%D2%90%D1%80%D0%B5%D0%BD%D0%BB%D0%B0%D0%BD%D0%B4%D1%81%D1%8C%D0%BA%D0%B5_%D0%BC%D0%BE%D1%80%D0%B5&action=edit&section=0'
a = 'https://uk.wikipedia.org/w/index.php?title=%D2%90%D1%80%D0%B5%D0%BD%D0%BB%D0%B0%D0%BD%D0%B4%D1%81%D1%8C%D0%BA%D0%B5_%D0%BC%D0%BE%D1%80%D0%B5&action=edit&section=0'
r = requests.get(url)
html = BeautifulSoup(r.content, 'lxml')
text = re.search(r'({{.*}})', html.find('textarea').get_text(), re.DOTALL)

In [2]:
def get_card(url):
    r = requests.get(url)
    html = BeautifulSoup(r.content, 'lxml')
    text = re.search(r'({{.*}})', html.find('textarea').get_text(), re.DOTALL)
    if text:
        return text.group(0).split('\n')

def find_real_title(url):
    r = requests.get(url)
    tmp = r.content.replace(b'<link rel="canonical" href="', b'r@ndom}-=||').split(b'r@ndom}-=||')[-1]
    idx = tmp.find(b'"/>')
    real_link = tmp[:idx].decode()
    return real_link.strip('https://uk.wikipedia.org/wiki/')
    
def clean(entry):
    res = entry.strip()
    res = res.replace("''", '"').replace("'", "")
    res = re.sub(r'<.*?>', '', res)
    res = re.sub(r'{{.*?}}', '', res)
    res = res.replace('[[', '').replace(']]', '')
    res = re.sub(r'\[.*?\]', '', res)
    res = res.replace('&nbsp;', ' ')
    if res.count('|') == 1:
        spl = res.split('|')
        res = '{w1} ({w2})'.format(w1=spl[0].strip(),
                                   w2=spl[1].strip())
    if all((c.isdigit() or c in ' ,') for c in res):
        res = res.replace(' ', '').replace(',', '.')
        try:
            res = int(res)
        except:
            try:
                res = float(res)
            except:
                res = res
    return res

def parse_card(card):
    res_dict = OrderedDict()
    special_entries = []
    for line in card:
        if not line.strip().startswith('|'):
            continue
        if line.count('=') != 1:
            if ('lat' in line) and ('lon' in line):
                res_dict['coordinates'] = line
            special_entries.append(line)
        else:
            cat, entry = line.split('=')
            cat = cat.strip(' |')
            entry = clean(entry)
            res_dict[cat] = entry
    return res_dict

def populate_dict(info_dict, name, url_title, base_url):
    try:
        initial_url = 'https://uk.wikipedia.org/wiki/{title}'.format(title=url_title)
        real_title = find_real_title(initial_url)
        card = get_card(base_url.format(title=real_title))
        if not card:
            print('Could not get card for', name)
            return
    except:
        print('Problem with page', name)
        return
    time.sleep(0.1)
    parsed_card = parse_card(card)
    info_dict[name] = parsed_card
#parse_card(card)

Спершу дістаємо країни.

In [3]:
def scrape_countries(html):
    country_urls = []
    seen_urls = []
    body = html.find('div', {'id': 'bodyContent', 'class': 'mw-body-content'})
    for li in body.find_all('li'):
        for a in li.find_all('a'):
            link_text = a.get_text().strip()
            link_url = a.get('href')
            if link_url in seen_urls:
                continue
            seen_urls.append(link_url)
            if not link_text:
                continue
            if 'Країни з прапорцями' in link_text:
                return country_urls
            else:
                if (link_text[0] == link_text[0].upper() 
                    or 'остр' in link_text) and ('wiki' in link_url):
                    country_urls.append((link_url[6:], link_text))
                else:
                    continue
    return country_urls

In [ ]:
list_url = "https://uk.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D1%80%D0%B0%D1%97%D0%BD_%D1%81%D0%B2%D1%96%D1%82%D1%83"
r = requests.get(list_url)
html = BeautifulSoup(r.content, "lxml")
country_urls = scrape_countries(html)

In [6]:
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
country_dict = OrderedDict()
for url_title, name in tqdm(country_urls):
    populate_dict(country_dict, name, url_title, BASE_URL)

 77%|███████▋  | 236/307 [03:53<01:10,  1.01it/s]

Could not get card for Шпіцбергенським трактатом


100%|██████████| 307/307 [05:04<00:00,  1.01it/s]


In [5]:
allkeys = []
for c in country_dict:
    for k in country_dict[c]:
        allkeys.append(k)
allkeys = set(allkeys)

NameError: name 'country_dict' is not defined

In [18]:
keydict = {
    'Площа': ['area', 'area_km2', 'площа_території'],
    'Місце за площею': ['area_rank', 'класифікація_за_розміром_території'],
    'Густота населення': ['population_density', 'густота_заселення', 'population_density_km2'],
    'Місце за густотою населення': ['population_density_rank', 'класифікація_за_густотою_заселення'],
    'Населення': ['population_estimate', 'population'],
    'Місце за населенням': ['population_estimate_rank'],
    'Рік останнього перепису населення': ['population_census_year', 'рік_перепису_населення'],
    'Загальновживана назва': ['common_name', 'name', 'назва'],
    'Назва в родовому відмінку': ['common_name_g', 'назва_род', 'загальнопоширена_назва'],
    'Офіційна назва': ['conventional_long_name', 'офіційна_назва'],
    'Назва державною мовою': ['native_name', 'native_namekathy', 'місцева_назва', 'міцева_назва'],
    'Національний гімн': ['national_anthem', 'національний_гімн'],
    'Офіційні мови': ['official_languages', 'main_languages', 'languages', 'major_language',
                      'офіційні_мови', 'working_languages'],
    'Столиця': ['capital', 'столиця'],
    'Координати': ['coordinates'],
    'Найбільше місто': ['largest_city', 'найбільше_місто'],
    'Демонім': ['demonym'],
    'Форма правління': ['government_type', 'форма_правління'],
    'Імена лідерів': ['імена_лідерів_країни', 'leader_names'],
    'Посади лідерів': ['головні_керівні_посади', 'leader_titles'],
    'Лідери': ['leaders'],
    'Відсоток, покритий водою': ['percent_water', 'покрито_водою_відсоток'],
    'ВВП (ПКС)': ['GDP_PPP', 'ВВП_ПКС', 'GDP'],
    'ВВП (ПКС) на душу населення': ['GDP_PPP_per_capita', 'GDP_per_capita',
                           'ВВП_ПКС_на_душу_населення', 'ВВП_на_душу_населення'],
    'Місце за ВВП (ПКС)': ['GDP_PPP_rank', 'GDP_rank', 'класифікація_за_рівнем_ВВП',
                    'класифікація_за_рівнем_ВВП_ПКС'],
    'Індекс розвитку (HDI)': ['HDI'],
    'Місце за індексом розвитку (HDI)': ['HDI_rank'],
    'Валюта': ['currency', 'валюта'],
    'Код валюти': ['currency_code', 'код_валюти'],
    'Часовий пояс': ['time_zone', 'часовий_пояс'],
    'UTC-поправка': ['utc_offset', 'utc_поправка'],
    'Інтернет-домен': ['cctld', 'інтернет_домен'],
    'Телефонний код': ['calling_code', 'телефонний_код'],
    'Ключові події в історії': ['established_events', 'незалежність'],
    'Ключові дати в історії': ['established_dates', 'дати_отримання_незалежності'],
    'Ключові події і дати в історії': ['events_dates']
}

In [5]:
def get_coord(raw_coord):
    clist = [c.strip() for c in clean(raw_coord).strip(' |').split('|')
            if '=' in c]
    #print(clist)
    replace_dict = {'lat_dir': 'latNS', 'lon_dir': 'longEW',
                    'lat_deg': 'latd', 'lat_min': 'latm',
                    'lon_deg': 'longd', 'lon_min': 'longm'}
    for c in clist:
        if c in replace_dict.keys():
            c.replace(c, replace_dict[c])
    coord_dict = dict()
    for c in clist:
        if not len(c.split('=')) == 2:
            return 'Помилка в даних: {coords}'.format(coords = ' '.join(clist))
        k, v = c.split('=')
        coord_dict[k.strip()] = v.strip()
    if not 'latd' in coord_dict.keys():
        return 'Помилка в даних: {coords}'.format(coords = ' '.join(clist))
    directions = {"N": "північної",
                  "S": "південної",
                  "E": "східної",
                  "W": "західної"}
    try:
        latd = coord_dict['latd']
        latm = coord_dict['latm']
        lat_dir = directions[coord_dict['latNS']]
        lond = coord_dict['longd']
        lonm = coord_dict['longm']
        lon_dir = directions[coord_dict['longEW']]
    except KeyError:
        return 'Помилка в даних: {coords}'.format(coords = ' '.join(clist))
    lat = "{latd}° {latm}′ {lat_dir} широти".format(latd=latd,
                                                    latm=latm, 
                                                    lat_dir=lat_dir)
    lon = "{lond}° {lonm}′ {lon_dir} довготи".format(lond=lond,
                                                    lonm=lonm, 
                                                    lon_dir=lon_dir)
    return lat+', '+lon

directions = {"N": "північної",
              "S": "південної",
              "E": "східної",
              "W": "західної",
              "": ""}

In [6]:
def fix_coordinates(info_dict):
    for c in info_dict:
        if ('longEW' in info_dict[c].keys() and 'latm' in info_dict[c].keys()
            and 'latd' in info_dict[c].keys()):
            latd = info_dict[c]['latd']
            latm = info_dict[c]['latm']
            lat_dir = directions[info_dict[c]['latNS']]
            lond = info_dict[c]['longd']
            lonm = info_dict[c]['longm']
            lon_dir = directions[info_dict[c]['longEW']]
            lat = "{latd}° {latm}′ {lat_dir} широти".format(latd=latd,
                                                    latm=latm, 
                                                    lat_dir=lat_dir)
            lon = "{lond}° {lonm}′ {lon_dir} довготи".format(lond=lond,
                                                    lonm=lonm, 
                                                    lon_dir=lon_dir)
            coord = lat+', '+lon
            info_dict[c]['coordinates'] = coord
    return info_dict

In [7]:
def fix_leaders(info_dict):
    for c in info_dict:
        titles = [info_dict[c][k] for k in info_dict[c].keys() if re.search(r'leader_title[0-9]', k)]
        leaders = [info_dict[c][k] for k in info_dict[c].keys() if re.search(r'leader_name[0-9]', k)]
        if not len(titles) == len(leaders):
            continue
        key_t = 'leader_titles'
        key_l = 'leader_names'
        key_tl = 'leaders'
        info_dict[c][key_t] = ', '.join(titles)
        info_dict[c][key_l] = ', '.join(leaders)
        info_dict[c][key_tl] = ', '.join('{t} - {n}'.format(t=t, n=n) for (t, n) in zip(titles, leaders))
    return info_dict

In [8]:
def fix_events(info_dict):
    for c in info_dict:
        events = [info_dict[c][k] for k in info_dict[c].keys() if re.search(r'established_event[0-9]', k)]
        dates = [str(info_dict[c][k]) for k in info_dict[c].keys() if re.search(r'established_date[0-9]', k)]
        if not len(events) == len(dates):
            continue
        key_e = 'established_events'
        key_d = 'established_dates'
        key_ed = 'events_dates'
        info_dict[c][key_e] = ', '.join(events)
        info_dict[c][key_d] = ', '.join(dates)
        info_dict[c][key_ed] = ', '.join('{t} - {n}'.format(t=t, n=n) for (t, n) in zip(events, dates))
    return info_dict

In [9]:
def standardize_keys(info_dict, keydict):
    for k, v in keydict.items():
        for c in info_dict:
            for oldk in v:
                if oldk in info_dict[c].keys():
                    info_dict[c][k] = info_dict[c][oldk]
                    break
    for c in info_dict:
        info_dict[c] = {k:v for k,v in info_dict[c].items() if k in keydict}
    for c in info_dict:
        for k in info_dict[c]:
            if info_dict[c][k] == '':
                info_dict[c][k] = 'нема інформації в базі'
    return info_dict

In [14]:
country_dict = fix_coordinates(country_dict)
country_dict = fix_events(country_dict)
country_dict = fix_leaders(country_dict)
country_dict = standardize_keys(country_dict, keydict)

In [11]:
#with open('countries.pkl', 'wb') as wf:
#    pickle.dump(country_dict, wf)

with open('countries.pkl', 'rb') as f:
    country_dict = pickle.load(f)

<hr>

Міста: спершу якомога більше міст із окремих списків, потім - враховуючи неминучі втрати деяких міст - окремо міста-мільйонники і столиці.

In [16]:
city_url = 'https://uk.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%A1%D0%BF%D0%B8%D1%81%D0%BA%D0%B8_%D0%BC%D1%96%D1%81%D1%82_%D0%B7%D0%B0_%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D0%BE%D1%8E'
r = requests.get(city_url)
html = BeautifulSoup(r.content, "lxml")

In [17]:
BASE_WIKI = 'https://uk.wikipedia.org'
city_urls = []
for a in tqdm(html.find_all('a')):
    if 'Міста' in a.get_text():
        city_list_url = a.get('href')
        list_r = requests.get(BASE_WIKI+city_list_url)
        time.sleep(0.2)
        list_html = BeautifulSoup(list_r.content, 'lxml')
        body = list_html.find('div', {'id': 'bodyContent', 'class': 'mw-body-content'})
        if not body:
            print('Не знаходяться:', a.get_text())
        entries = (body.find_all('li') + body.find_all('td'))
        if not entries:
            print('Не знаходяться:', a.get_text())
        for entry in (body.find_all('li') + body.find_all('td')):
            city_link = entry.find('a')
            if not city_link:
                continue
            city_url = city_link.get('href')
            if (not city_url) or (not '/wiki/' in city_url):
                continue
            city_name = city_link.get_text()
            if not city_name or 'pt:' in city_name or city_name.startswith('Спис'):
                continue
            city_urls.append((city_name, city_url[6:]))

100%|██████████| 246/246 [01:19<00:00,  3.09it/s]


In [ ]:
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
city_info_dict = OrderedDict()
past_urls = []
for name, url_title in tqdm(city_urls):
    if name.startswith('Міста ') or name.startswith('Спис'):
        continue
    if name == '':
        continue
    if url_title in past_urls:
        continue
    past_urls.append(url_title)
    if 'pt:' in name:
        continue
    populate_dict(city_info_dict, name, url_title, BASE_URL)
    #if not any(('населення' in k.lower() or 'population' in k.lower()) 
    #           for k in city_info_dict[name]):
    #    continue

In [19]:
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
million_url = 'https://uk.wikipedia.org/wiki/%D0%9C%D1%96%D1%81%D1%82%D0%B0-%D0%BC%D1%96%D0%BB%D1%8C%D0%B9%D0%BE%D0%BD%D0%BD%D0%B8%D0%BA%D0%B8_%D1%81%D0%B2%D1%96%D1%82%D1%83'
rm = requests.get(million_url)
msoup = BeautifulSoup(rm.content, 'lxml')
mtable = msoup.find_all('table')[1]
for row in tqdm(mtable.find_all('tr')[1:]):
    col = row.find_all('td')
    name = col[0].get_text()
    name = re.sub('\[.*\]', '', name)
    name = re.sub('\(.*\)', '', name)
    name = name.strip(' *')
    city_url = col[0].find('a').get('href')[6:]
    populate_dict(city_info_dict, name, city_url, BASE_URL)

 37%|███▋      | 142/385 [02:15<03:51,  1.05it/s]

Could not get card for Медан


100%|██████████| 385/385 [06:07<00:00,  1.05it/s]


In [20]:
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
capitals_url = 'https://uk.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%81%D1%82%D0%BE%D0%BB%D0%B8%D1%86%D1%8C_%D0%BA%D1%80%D0%B0%D1%97%D0%BD_%D1%81%D0%B2%D1%96%D1%82%D1%83'
rc = requests.get(capitals_url)
csoup = BeautifulSoup(rc.content, 'lxml')
ctable = csoup.find_all('table')[0]
for row in tqdm(ctable.find_all('tr')[1:]):
    col = row.find_all('td')
    name = col[1].get_text()
    name = re.sub('\[.*\]', '', name)
    name = re.sub('\(.*\)', '', name)
    name = name.strip(' *')
    city_url = col[1].find('a').get('href')[6:]
    populate_dict(city_info_dict, name, city_url, BASE_URL)

100%|██████████| 194/194 [02:59<00:00,  1.08it/s]


In [21]:
city_allkeys = []
for c in city_info_dict:
    for k in city_info_dict[c]:
        city_allkeys.append(k)
city_allkeys = Counter(city_allkeys)

In [13]:
keydict.keys()

dict_keys(['Площа', 'Місце за площею', 'Густота населення', 'Місце за густотою населення', 'Населення', 'Місце за населенням', 'Рік останнього перепису населення', 'Загальновживана назва', 'Назва в родовому відмінку', 'Офіційна назва', 'Назва державною мовою', 'Національний гімн', 'Офіційні мови', 'Столиця', 'Координати', 'Найбільше місто', 'Демонім', 'Форма правління', 'Імена лідерів', 'Посади лідерів', 'Лідери', 'Відсоток, покритий водою', 'ВВП (ПКС)', 'ВВП (ПКС) на душу населення', 'Місце за ВВП (ПКС)', 'Індекс розвитку (HDI)', 'Місце за індексом розвитку (HDI)', 'Валюта', 'Код валюти', 'Часовий пояс', 'UTC-поправка', 'Інтернет-домен', 'Телефонний код', 'Ключові події в історії', 'Ключові дати в історії', 'Ключові події і дати в історії'])

In [19]:
keydict['Площа'].extend(['area_municipality', 'area_urban_km2', 'area_total', 'area_metro',
                         'area_urban', 'area_metro_km2', 'area_total_km2', "площа", "площадь",
                         'TotalArea', "Площа"])
keydict['Густота населення'].extend(["густота", "густота", "густота населення",
                                     "pop_dens", 'population_density_urban_km2',
                                     "Густота", 'Density', 'density_km2',
                                     'Щільність населення'])
keydict['Населення'].extend(['metro_pop', 'pop-total', 'pop_municipality', 'population_(Metro)',
                             'population_est', 'population_metro', 'population_total',
                             'population_urban', 'Населення', 'Population', "населення_згідно_перепису",
                             "повне_населення"])
keydict['Загальновживана назва'].extend(['Назва', 'Неофіційна назва',
                                         'загальнопоширена_назва', 'назва українською',
                                         'українська назва', 'Назва українською', 'Name',
                                         "назва укр."])
keydict['Назва в родовому відмінку'].extend(['name_g', "назва в родовому відмінку",
                                             "Родовий відмінок"])
keydict['Назва державною мовою'].extend(['Місцева назва', "місцева назва", "Назва оригінальна",
                                         "Оригінальна назва", "назва мовою оригіналу",
                                         "самоназва", "оригінальна назва", 'name_local'])
keydict['ВВП (ПКС)'].extend(['ввп'])
keydict['Телефонний код'].extend(["телефонний код"])
keydict['Країна'] = ["країна", "Країна"]
keydict['Регіон'] = ["регіон", "Регіон", 'region', 'Region']
keydict['Засноване'] = ['засноване', 'заснований', 'засновано', 'established_date']
keydict['Міста-побратими'] = ['міста-побратими', "міста побратими", 'sister-cities']
keydict['Висота над рівнем моря'] = ['elevation_m', 'elevation', 'elevation_max',
                                    "висота над рівнем моря", "висота", "середня висота",
                                    "висота центру НП"]
keydict['Веб-сторінка'] = ['веб-сторінка', 'вебсайт', 'веб-сайт', 'website']
keydict['Мер'] = ['mayor', 'Mayor', "мер", "Мер", "голова"]
keydict['Девіз'] = ['девіз', 'motto']
keydict['Поділ міста'] = ['поділ міста', 'поділ', "внутрішній поділ", "внутрішній розподіл"]

In [25]:
#city_info_dict = fix_coordinates(city_info_dict)
city_info_dict = fix_events(city_info_dict)
city_info_dict = fix_leaders(city_info_dict)
city_info_dict = standardize_keys(city_info_dict, keydict)

In [16]:
new_city_dict = dict()
for k in list(city_info_dict.keys()):
    if k in country_dict.keys():
        continue
    else:
        new_city_dict[k] = city_info_dict[k]
city_info_dict = new_city_dict

In [20]:
#with open('cities.pkl', 'wb') as wf:
#    pickle.dump(city_info_dict, wf)

with open('cities.pkl', 'rb') as f:
    city_info_dict = pickle.load(f)

In [13]:
city_info_dict['Варшава']

{'Площа': 'нема інформації в базі',
 'Населення': 3370000,
 'Загальновживана назва': 'Варшава',
 'Назва державною мовою': 'Warszawa',
 'Імена лідерів': 'нема інформації в базі',
 'Посади лідерів': 'нема інформації в базі',
 'Лідери': 'нема інформації в базі',
 'Ключові події в історії': 'нема інформації в базі',
 'Ключові дати в історії': 'нема інформації в базі',
 'Ключові події і дати в історії': 'нема інформації в базі',
 'Країна': 'нема інформації в базі',
 'Регіон': 'Файл:POL województwo mazowieckie flag.svg (25px Мазовецьке воєводство)',
 'Засноване': '13 століття',
 'Міста-побратими': 'нема інформації в базі',
 'Висота над рівнем моря': '78-115',
 'Веб-сторінка': 'нема інформації в базі',
 'Мер': 'Ганна Гронкевич-Вальц',
 'Девіз': '"Contemnit procellas" (Кидає виклик бурям)"Semper invicta" (Ніколи неприборкана)',
 'Поділ міста': '18 районів'}

<hr>

Моря.

In [151]:
sea_url = 'https://uk.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BC%D0%BE%D1%80%D1%96%D0%B2'
sea_r = requests.get(sea_url)
sea_soup = BeautifulSoup(sea_r.content, 'lxml')
oceans = sea_soup.find_all('th')
sea_groups = sea_soup.find_all('td')[:8]
seas_lists = [s.find_all('li') for s in sea_groups]
seas = [item for sublist in seas_lists for item in sublist]

In [152]:
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
sea_info_dict = OrderedDict()
for o in oceans:
    name = o.find('a').get_text().strip()
    url_title = o.find('a').get('href')[6:]
    populate_dict(sea_info_dict, name, url_title, BASE_URL)
for sea in tqdm(seas):
    name = sea.find('a').get_text().strip()
    url_title = sea.find('a').get('href')[6:]
    populate_dict(sea_info_dict, name, url_title, BASE_URL)

100%|██████████| 107/107 [01:34<00:00,  1.13it/s]


In [14]:
def fix_oceans(info_dict):
    for ocean in list(info_dict.keys()):
        if not 'океан' in ocean:
            continue
        for k in list(info_dict[ocean].keys()):
            if 'мітка' in k:
                mnum = k[-1]
                new_key = info_dict[ocean][k]
                try:
                    text = info_dict[ocean]['текст'+mnum]
                    if not text: continue
                    info_dict[ocean][new_key] = info_dict[ocean]['текст'+mnum]
                except:
                    continue
    return info_dict

In [154]:
sea_info_dict = fix_oceans(sea_info_dict)

In [155]:
seakeys = []
for k in sea_info_dict:
    seakeys.extend(list(sea_info_dict[k].keys()))
seakeys = list(set(seakeys))

In [11]:
keydict['Найбільша глибина'] = ['Найбільша глибина', 'Максимальна глибина', "максимальна глибина",
                                "максимальна_глибина", "найбільша глибина", "max-depth"]
keydict['Середня глибина'] = ['Середня глибина', 'сер глибина', "середня глибина", 
                              "середня_глибина", "depth"]
keydict['Розташування'] = ['Розташування', 'розташування', 'location']
keydict["Об'єм"] = ['Обєм', "об'єм", "Об'єм", 'volume']
keydict['Солоність'] = ['Солоність', "солоність", 'salinity']
keydict['Впадаючі річки'] = ['Витікаючі річки', 'Впадають річки', 'впадаючі річки',
                             'Впадаючі річки', 'впадаюча річка', "вливаються", "впадають"]
keydict['Довжина берегової лінії'] = ['Довжина берегової лінії', 'довжина берегової лінії',
                                      'берегова_лінія', 'берегова лінія']
keydict['Моря'] = ['Моря']
keydict['Океан'] = ['океан']
keydict['Країни'] = ['країни', "країна"]
keydict['Площа водозабору'] = ['Площа водозбору', 'площа водозабору', 'Площа водозабору']
keydict['Прозорість'] = ['Прозорість', "прозорість"]

In [167]:
sea_info_dict = standardize_keys(sea_info_dict, keydict)

In [16]:
#with open('seas.pkl', 'wb') as wf:
#    pickle.dump(sea_info_dict, wf)

with open('seas.pkl', 'rb') as f:
    sea_info_dict = pickle.load(f)

<hr>

In [159]:
isl_url = "https://uk.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B2%D1%96%D0%B2_%D0%B7%D0%B0_%D0%BF%D0%BB%D0%BE%D1%89%D0%B5%D1%8E"
isr = requests.get(isl_url)
isl_soup = BeautifulSoup(isr.content, 'lxml')
tables = isl_soup.find_all('table')
isl_info_dict = OrderedDict()
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
for table in tables[1:-1]:
    for row in tqdm(table.find_all('tr')[1:]):
        col = row.find_all('td')
        name = col[1].get_text()
        url_title = col[1].find('a').get('href')[6:]
        populate_dict(isl_info_dict, name, url_title, BASE_URL)

 81%|████████▏ | 26/32 [00:21<00:04,  1.20it/s]

Could not get card for Ямайка


  8%|▊         | 5/60 [00:05<00:55,  1.01s/it]

Could not get card for Ломбок


 65%|██████▌   | 39/60 [00:36<00:19,  1.06it/s]

Could not get card for Сан-Крістобаль


100%|██████████| 60/60 [00:55<00:00,  1.08it/s]


In [160]:
islkeys = []
for k in isl_info_dict:
    islkeys.extend(list(isl_info_dict[k].keys()))
islkeys = list(set(islkeys))

In [12]:
keydict['Розташування'].extend(['Місце'])
keydict['Площа'].extend(['Загальна площа', "Площа\xa0"])
keydict['Група островів'] = ['Група', "група"]

In [170]:
isl_info_dict = standardize_keys(isl_info_dict, keydict)

In [18]:
#with open('isls.pkl', 'wb') as wf:
#    pickle.dump(isl_info_dict, wf)

with open('isls.pkl', 'rb') as f:
    isl_info_dict = pickle.load(f)

In [19]:
isl_info_dict['Сардинія']

{'Площа': 24100,
 'Загальновживана назва': 'Сардинія',
 'Назва державною мовою': 'нема інформації в базі',
 'Найбільше місто': 'Кальярі',
 'ВВП (ПКС)': '33.6 млрд. євро',
 'Часовий пояс': 'UTC+1 (взимку),UTC+2 (влітку)',
 'Країна': 'нема інформації в базі',
 'Регіон': 'Кальярі (провінція)|Кальярі,Карбонія-Іглезіас (провінція)|Карбонія-Іглезіас,Медіо-Кампідано (провінція)|Медіо-Кампідано,Нуоро (провінція)|Нуоро,Ольястра (провінція)|Ольястра,Ольбія-Темпіо (провінція)|Ольбія-Темпіо,Орістано (провінція)|Орістано,Сассарі (провінція)|Сассарі',
 'Засноване': '26 лютого 1948(надано статус автономії)',
 'Веб-сторінка': 'http://www.regione.sardegna.it/ www.regione.sardegna.it',
 'Розташування': 'Regione Sardegna 3.svg',
 'Країни': 'нема інформації в базі'}

<hr>

Річки

In [47]:
river_url = "https://uk.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%80%D1%96%D1%87%D0%BE%D0%BA_%D0%B7%D0%B0_%D0%B4%D0%BE%D0%B2%D0%B6%D0%B8%D0%BD%D0%BE%D1%8E"
rr = requests.get(river_url)
rsoup = BeautifulSoup(rr.content, 'lxml')

In [48]:
river_info_dict = OrderedDict()
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
table = rsoup.find_all('table')[1]
for row in tqdm(table.find_all('tr')[1:]):
    col = row.find_all('td')
    rivers = col[1].find_all('a')
    for river in rivers:
        name = river.get_text()
        url_title = river.get('href')[6:]
        populate_dict(river_info_dict, name, url_title, BASE_URL)

 98%|█████████▊| 171/174 [03:04<00:03,  1.08s/it]

Could not get card for Ессекібо


100%|██████████| 174/174 [03:07<00:00,  1.08s/it]


In [49]:
rivkeys = []
for k in river_info_dict:
    rivkeys.extend(list(river_info_dict[k].keys()))
rivkeys = list(set(rivkeys))

In [53]:
keydict['Довжина'] = ['довжина']
keydict['Розташування гирла'] = ['витік коорд', "гирло_коорд", "гирло коорд", "витік_коорд"]
keydict['Гирло']  = ['гирло']
keydict['Площа басейну'] = ['площа басейну', 'басейн', "площа_басейну"]
keydict['Країни басейну'] = ['країни басейну', 'країни_басейну', 'прирічкові_країни', 'прирічкові країни']
keydict['Стік'] = ['стік', 'максимальний_стік']
keydict['Притоки'] = ['притоки']

In [54]:
river_info_dict = standardize_keys(river_info_dict, keydict)

In [55]:
#with open('train_data/rivers.pkl', 'wb') as wf:
#    pickle.dump(river_info_dict, wf)

with open('train_data/rivers.pkl', 'rb') as f:
    river_info_dict = pickle.load(f)

<hr>

Озера.

In [22]:
lake_url = "https://uk.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BD%D0%B0%D0%B9%D0%B1%D1%96%D0%BB%D1%8C%D1%88%D0%B8%D1%85_%D0%BE%D0%B7%D0%B5%D1%80_%D1%81%D0%B2%D1%96%D1%82%D1%83"
lake_r = requests.get(lake_url)
lake_soup = BeautifulSoup(lake_r.content, 'lxml')

In [23]:
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
lake_info_dict = OrderedDict()
table = lake_soup.find_all('table')[1]
for row in tqdm(table.find_all('tr')[1:]):
    col = row.find_all('td')
    lake = col[0].find('a')
    name = lake.get_text()
    url_title = lake.get('href')[6:]
    populate_dict(lake_info_dict, name, url_title, BASE_URL)

 33%|███▎      | 29/87 [00:31<01:02,  1.07s/it]

Could not get card for Женевське озеро


100%|██████████| 87/87 [01:39<00:00,  1.14s/it]


In [24]:
lakeys = []
for k in lake_info_dict:
    lakeys.extend(list(lake_info_dict[k].keys()))
lakeys = list(set(lakeys))

In [27]:
[k for k in lakeys if 'typ' in k.lower()]

['type', 'region:EU_type:waterbody|display', 'lake_type']

In [14]:
keydict["Загальновживана назва"].extend(['lake_name'])
keydict['Ширина'] = ['ширина', 'width']
keydict['Середня глибина'].extend(['глибина'])
keydict["Країни"].extend(['countries', 'basin_countries', "країни_басейну", "країни басейну"])
keydict["Міста"] = ['cities', 'major cities', "міста"]
keydict["Впадаючі річки"].extend(['витікають'])
keydict["Тип"] = ['тип', 'type', 'Тип', 'lake_type']

In [29]:
lake_info_dict = standardize_keys(lake_info_dict, keydict)

In [31]:
#with open('lakes.pkl', 'wb') as wf:
#    pickle.dump(lake_info_dict, wf)

with open('lakes.pkl', 'rb') as f:
    lake_info_dict = pickle.load(f)

In [32]:
lake_info_dict['Балхаш']

{'Площа': 16400,
 'Загальновживана назва': 'Балха́ш',
 'Назва державною мовою': 'нема інформації в базі',
 'Висота над рівнем моря': 340,
 'Найбільша глибина': 26,
 'Середня глибина': 5.8,
 "Об'єм": 112,
 'Впадаючі річки': 'Ілі (ріка)|Ілі, Каратал (ріка)|Каратал, Аксу (притока Балхашу)|Аксу, Лепси (ріка)|Лепси, Аягуз (річка)|Аягуз',
 'Країни': 'Казахстан',
 'Площа басейну': 413000,
 'Країни басейну': 'Казахстан',
 'Міста': 'нема інформації в базі',
 'Тип': 'нема інформації в базі'}

<hr>

Гори.

In [33]:
mont_url = "https://uk.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D0%B5%D0%BB%D1%96%D0%BA_%D0%BA%D1%80%D0%B0%D1%97%D0%BD_%D0%B7%D0%B0_%D0%BD%D0%B0%D0%B9%D0%B2%D0%B8%D1%89%D0%B8%D0%BC%D0%B8_%D1%82%D0%BE%D1%87%D0%BA%D0%B0%D0%BC%D0%B8"
mr = requests.get(mont_url)
msoup = BeautifulSoup(mr.content, 'lxml')

In [43]:
BASE_URL = 'https://uk.wikipedia.org/w/index.php?title={title}&action=edit&section=0'
mont_info_dict = OrderedDict()
table = msoup.find_all('table')[0]
for row in tqdm(table.find_all('tr')[1:]):
    col = row.find_all('td')
    mont = col[2].find('a')
    if not mont:
        continue
    name = mont.get_text()
    url_title = mont.get('href')[6:]
    populate_dict(mont_info_dict, name, url_title, BASE_URL)

 29%|██▉       | 57/197 [00:57<02:20,  1.01s/it]

Could not get card for Кіньєті


 35%|███▍      | 68/197 [01:09<02:11,  1.02s/it]

Could not get card for [5]


100%|██████████| 197/197 [03:24<00:00,  1.04s/it]


In [44]:
mont_url2 = "https://uk.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D1%96%D1%80_%D0%B7%D0%B0_%D0%B2%D1%96%D0%B4%D0%BD%D0%BE%D1%81%D0%BD%D0%BE%D1%8E_%D0%B2%D0%B8%D1%81%D0%BE%D1%82%D0%BE%D1%8E"
mr = requests.get(mont_url2)
msoup = BeautifulSoup(mr.content, 'lxml')

In [ ]:
table = msoup.find_all('table')[0]
for row in tqdm(table.find_all('tr')[1:]):
    col = row.find_all('td')
    mont = col[1].find('a')
    if not mont:
        continue
    name = mont.get_text()
    url_title = mont.get('href')[6:]
    populate_dict(mont_info_dict, name, url_title, BASE_URL)

In [50]:
mont_url3 = "https://uk.wikipedia.org/wiki/%D0%93%D1%96%D1%80%D1%81%D1%8C%D0%BA%D0%B0_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0"
mr = requests.get(mont_url3)
msoup = BeautifulSoup(mr.content, 'lxml')
table = msoup.find_all('table')[0]
for row in tqdm(table.find_all('tr')[1:]):
    col = row.find_all('td')
    mont = col[0].find('a')
    if not mont:
        continue
    name = mont.get_text()
    url_title = mont.get('href')[6:]
    populate_dict(mont_info_dict, name, url_title, BASE_URL)


100%|██████████| 22/22 [00:21<00:00,  1.00it/s]

Could not get card for Памір


In [51]:
monkeys = []
for k in mont_info_dict:
    monkeys.extend(list(mont_info_dict[k].keys()))
monkeys = list(set(monkeys))

In [15]:
keydict['Тип'].extend(['Type'])
keydict['Гірська система'] = ['гірська_система', 'система']
keydict['Відносна висота'] = ['відносна_висота', "висота відносна", "відносна висота"]
keydict['Висота над рівнем моря'].extend(['найвища_висота', 'Elevation', 'highest_elevation'])
keydict['Перше сходження'] = ['перше_сходження', 'сходження']
keydict['Вік'] = ['Age', 'age', "вік", 'період', 'період1']
keydict['Гороутворення'] = ['гороутворення', 'гороутворення1']

In [75]:
mont_info_dict = standardize_keys(mont_info_dict, keydict)

In [76]:
#with open('mountains.pkl', 'wb') as wf:
    pickle.dump(mont_info_dict, wf)

with open('mountains.pkl', 'rb') as f:
    mont_info_dict = pickle.load(f)

In [77]:
mont_info_dict['Говерла']

{'Загальновживана назва': 'Говерла',
 'Країна': 'нема інформації в базі',
 'Висота над рівнем моря': '2061 м',
 'Країни': 'нема інформації в базі',
 'Тип': 'нема інформації в базі',
 'Гірська система': 'Чорногора, Українські Карпати, Карпати',
 'Перше сходження': 'Невідомо',
 'Вік': 'нема інформації в базі'}

<hr>

In [59]:
unit_dict = {'Площа': "км²",
 'Місце за площею': 'у світі',
 'Густота населення': 'осіб на км²',
 'Місце за густотою населення': "у світі",
 'Населення': "осіб",
 'Місце за населенням': "у світі",
 'Рік останнього перепису населення': "",
 'Загальновживана назва': "",
 'Назва в родовому відмінку': "",
 'Офіційна назва': "",
 'Назва державною мовою': "",
 'Національний гімн': "",
 'Офіційні мови': "",
 'Столиця': "",
 'Координати': "",
 'Найбільше місто': "",
 'Демонім': "",
 'Форма правління': "",
 'Імена лідерів': "",
 'Посади лідерів': "",
 'Лідери': "",
 'Відсоток, покритий водою': "%",
 'ВВП (ПКС)': "доларів",
 'ВВП (ПКС) на душу населення': "доларів",
 'Місце за ВВП (ПКС)': "у світі",
 'Індекс розвитку (HDI)': "",
 'Місце за індексом розвитку (HDI)': "у світі",
 'Валюта': "",
 'Код валюти': "",
 'Часовий пояс': "",
 'UTC-поправка': "год",
 'Інтернет-домен': "",
 'Телефонний код': "",
 'Ключові події в історії': "",
 'Ключові дати в історії': "",
 'Ключові події і дати в історії': "",
 'Країна': "",
 'Регіон': "",
 'Засноване': "",
 'Міста-побратими': "",
 'Висота над рівнем моря': "м",
 'Веб-сторінка': "",
 'Мер': "",
 'Девіз': "",
 'Поділ міста': "",
 'Найбільша глибина': "м",
 'Середня глибина': "м",
 'Розташування': "",
 "Об'єм": "м³",
 'Солоність': "‰",
 'Впадаючі річки': "",
 'Площа басейну': "км²",
 'Довжина берегової лінії': "км",
 'Моря': "",
 'Океан': "",
 'Площа водозабору': "км²",
 'Прозорість': "",
 'Група островів': "",
 'Країни': "",
 'Ширина': "км",
 'Міста': "",
 'Тип': "",
 'Гірська система': "",
 'Відносна висота': "м",
 'Перше сходження': "",
 'Вік': "",
 'Гороутворення': ""}

with open('data/units.pkl', 'wb') as f:
    pickle.dump(unit_dict, f)

<hr>

In [ ]:
dict_df['repl'] = dict_df[1].str.replace('\(.*\)', '')
dict_df[dict_df[1].duplicated()]

In [21]:
import glob
keys = []
for f in glob.glob('dicts/*'):
    dicname = f[5:-4]
    dic = pickle.load(open(f, 'rb'))
    for k in dic:
        try:
            keys.append((dicname, k, len(dic[k].keys())))
        except:
            print(k)
            continue
dict_df = pd.DataFrame(keys)
disamb_dict = dict(zip(list(dict_df[0].value_counts().index),
     ['місто/регіон', "країна", "гора", "річка", "острів", "море", "озеро"]))

In [23]:
len(dict_df)

9001

In [48]:
unified_dict = dict()
total_len = 0
for f in glob.glob('data/*')[:-1]:
    dicname = f[5:-4]
    dic = pickle.load(open(f, 'rb'))
    total_len += len(dic)
    new_dict = dict()
    for k in list(dic.keys()):
        old_k = k
        if '(' in k:
            detail = re.search(r'\((.*?)\)', k).group(1)
            k = re.sub(r'\(.*?\)', '', k).strip()
            if k in no_touch:
                k = old_k
        else:
            detail = None
        if k in list(dict_df[dict_df[1].duplicated()][1]) or k in new_dict:
            new_k = k + ' [{dis}]'.format(dis=disamb_dict[dicname])
        else:
            new_k = k
        if new_k in new_dict:
            print(new_k)
        new_dict[new_k] = dic[old_k]
        if detail:
            new_dict[new_k]['уточнення'] = detail
    unified_dict.update(new_dict)

Північна Ютландія [місто/регіон]


In [52]:
unified_dict['Дніпро [річка]']

{'Загальновживана назва': 'Дніпро',
 'Середня глибина': '6-12 м',
 'Країни': 'нема інформації в базі',
 'Розташування гирла': 'нема інформації в базі',
 'Гирло': 'Дніпровсько-Бузький лиман, Чорне море',
 'Площа басейну': '504 300 км²',
 'Країни басейну': 'нема інформації в базі',
 'Стік': '1670 м³/с',
 'Притоки': '"Праві"'}

In [ ]:
new_dict = dict()
for k in unified_dict:
    if 'ґ' in k:
        new_k = k.replace('ґ', 'г')
    else:
        new_k = k
    new_dict[new_k] = unified_dict[k]
    
unified_dict = new_dict

In [56]:
for k in unified_dict:
    if k.replace(' [річка]', '') in river_info_dict:
        if 'Довжина' in river_info_dict[k.replace(' [річка]', '')]:
            unified_dict[k]['Довжина'] = river_info_dict[k.replace(' [річка]', '')]['Довжина']

In [57]:
#with open('data/obj_dict.pkl', 'wb') as f:
#    pickle.dump(unified_dict, f)

with open('data/obj_dict.pkl', 'rb') as f:
    unified_dict = pickle.load(f)

<hr>

In [26]:
traindf = pd.read_csv('train3.csv')

In [35]:
def fill_blank(q):
    
    
    res = q.replace('[', '').replace(']', '')
    for k in blank_dict:
        if k in res:
            repl = random.choice(blank_dict[k])
            res = res.replace(k, repl)
    return res

traindf['Q'] = traindf['Q'].apply(fill_blank)

In [44]:
cities = ['Варшава', "Рига", "Мумбай", "Малабо", "Ріо-де-Жанейро"]
seas = ['Чорне море', "Балтійське море"]
lakes = ["Балатон", "Тітікака"]
rivers = ["Дніпро", "Амазонка", "Янцзи"]
monts = ["Риси", "Говерла", "Аконкагуа", "Кіліманджаро", "Гімалаї"]
islands = ["Тасманія", "Сардинія", "Корсика", "Куба", "Ява"]

blank_dict = {
        '[city]': cities,
        '[sea]': seas,
        '[lake]': lakes,
        '[river]': rivers,
        '[mont]': monts,
        '[island]': islands
    }

lol = [cities, seas, lakes, rivers, monts, islands]
for k in blank_dict:
    for e in blank_dict[k]:
        pass
    
train2 = train2.sample(frac=3, replace=True)

In [45]:
train2.to_csv('train4.csv', index=False)

In [46]:
train1 = pd.read_csv('train2.csv')
train2 = pd.read_csv('train4.csv')
train = pd.concat([train1, train2])

In [39]:
train.to_csv('train_questions.csv', index=False)

In [ ]:
# create csv file for training
with open('train_q.txt') as f:
    train_q = f.read().split('\n')

In [ ]:
train_countries = ["Польща", "Ботсвана", "Екваторіальна Гвінея", "Австралія", "Алжир"]
lines = []
for c in train_countries:
    for q in train_q:
        line = q.replace('+++', c)+'\n'
        lines.append(line)
        
with open('train.csv', 'w') as wf:
    wf.writelines(lines)

In [ ]:
centries = []
for c in train_countries:
    for k in country_dict[c]:
        centries.append((k, country_dict[c][k]))

In [ ]:
tkeys = []
for c in train_countries:
    for k in country_dict[c]:
        tkeys.append(k)
        
allkeys = []
for c in country_dict:
    for k in country_dict[c]:
        allkeys.append(k)